In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import seaborn as sns
import pandas as pd
import plotly.express as px

from jupyter_dash import JupyterDash
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import matplotlib
from scipy import stats

from plotly.subplots import make_subplots
from numpy.random import normal
from numpy import hstack
from numpy import asarray
from numpy import exp
import matplotlib.pyplot as pyplot
import numpy
import scipy
from sklearn.neighbors import KernelDensity # generate a sample

from statistics import mean
import math
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import t
from scipy import stats

import dash_cytoscape as cyto
import networkx as nx

from plotly.subplots import make_subplots
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
df = pd.read_csv('US_IN_Country_data_copy.csv')
df1 = pd.read_csv('geomapsData.csv')
#df1.head(1)
#df.head(1)

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
#app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE])

#Graph 1 Drop Down Options
# Define the dropdown options
dropdown_options = [
    {'label': 'View Count', 'value': 'view_count'},
    {'label': 'Likes', 'value': 'likes'},
    {'label': 'Dislikes', 'value': 'dislikes'},
    {'label': 'Comment Count', 'value': 'comment_count'},
    {'label': 'Time to Trend (hours)', 'value': 'time_to_trend_hours'},
    {'label': 'Days to Trend', 'value': 'days_to trend'},
    {'label': 'Tags Count', 'value': 'tagsCount'},
    {'label': 'Title Length', 'value': 'titleLength'}
]

# Define the options for the year dropdown
year_options = [{'label': year, 'value': year} for year in df['year'].unique()]

#Define Country Options
country_options = [{'label': 'US', 'value': 'US'},{'label': 'IN', 'value': 'IN'}]

#Define Country Options
#country_options = [{'label': 'US', 'value': 'US'},{'label': 'IN', 'value': 'IN'}]

bin_options=[{"label": "100", "value": 100},{"label": "150", "value": 150},{"label": "200", "value": 200},]

bandwidth_options= [{"label": "0.1", "value": 0.1},{"label": "0.3 (optimal)", "value": 0.3},{"label": "0.5", "value": 0.5},]

country_options_contour = [{"label": "US", "value": 'US'},{"label": "IN", "value": 'IN'},{"label": "All", "value": None},]

# Define the categories to be used in the dropdown menu
categories = sorted(df['category'].unique())

# Define image paths
img_paths_wc = {
    'Tags-India': '/assets/wc_tags_IN.png',
    'Tags-USA': '/assets/wc_tags_US.png',
    'Title-India': '/assets/wc_title_IN.png',
    'Title-USA': '/assets/wc_title_US.png',
}
# Define dropdown options-wordcloud
dropdown_options_wc = [{'label': k, 'value': v} for k, v in img_paths_wc.items()]

In [ ]:
# filter out rows with tagCounts = 0
df_filter_tag0 = df[df['tagsCount'] > 10]

fig_dht = go.Figure(go.Histogram2d(
        x=df_filter_tag0['titleLength'],
        y=df_filter_tag0['tagsCount'],
        colorscale='agsunset', # Set the colorscale
        #coloraxis = "coloraxis",
        colorbar=dict(len=0.65,thickness=20,title='Counts')
    ))

fig_dht.add_trace(go.Histogram(
        x=df_filter_tag0['titleLength'],
        name='titleLength',
        nbinsx=50,
        #marker=dict(color='rgba(255, 0, 0, 0.7)'),
        marker=dict(color='#FF85B3'),
        opacity=0.75,
        yaxis='y2'
    ))

fig_dht.add_trace(go.Histogram(
        y=df_filter_tag0['tagsCount'],
        name='tagsCount',
        nbinsy=50,
        #marker=dict(color='rgba(0, 0, 255, 0.7)'),
        marker=dict(color='#FF85B3'),#PiYg #FF85B3
        opacity=0.75,
        xaxis='x2'
    ))

fig_dht.update_layout(
        xaxis=dict(domain=[0, 0.85], title='titleLength'),
        yaxis=dict(domain=[0, 0.85], title='tagsCount'),
        xaxis2=dict(domain=[0.85, 1], showgrid=False),
        yaxis2=dict(domain=[0.85, 1], showgrid=False),
        title='Density Heatmap: Title Length, Tags Count',
        height=400,
        width=600,
        plot_bgcolor='#313131',  # set background color of plot area
        paper_bgcolor='#313131',  # set background color of paper area (outside plot area)
        font=dict(color='#FF008E')
    )

In [ ]:
# Filter for 'Pets & Animals' category and 'US' country for NETWORK GRAPH Cytoscape
us_tags = df.loc[(df['category'] == 'Pets & Animals') & (df['country'] == 'US')]

# Extract tags column as list of strings
tags_list = us_tags['tags'].str.split('|').apply(lambda x: x[:5]).tolist()

# Create empty network graph
G1 = nx.Graph()

# Loop through each video's tags
for tags in tags_list:
    # Create nodes for each unique tag
    for tag in tags:
        if tag not in G1:
            G1.add_node(tag)
    # Create edges between tags that appear together in the same video
    for i in range(len(tags)):
        for j in range(i+1, len(tags)):
            if not G1.has_edge(tags[i], tags[j]):
                G1.add_edge(tags[i], tags[j], weight=1)
            else:
                G1[tags[i]][tags[j]]['weight'] += 1

# Convert networkx graph to cytoscape elements
elements1 = []
for node in G1.nodes:
    elements1.append({'data': {'id': node, 'label': node}})
for edge in G1.edges:
    elements1.append({'data': {'source': edge[0], 'target': edge[1], 'weight': G1.edges[edge]['weight']}})


In [ ]:
#Correalation Network Graph
data_NG = df[['views_log', 'likes_log', 'dislikes_log', 'comment_count_log', 'tagsCount', 'titleLength', 'time_to_trend_hours','days_to trend']]
data_NG = data_NG.rename(columns={'views_log': 'Views', 'likes_log': 'Likes', 'dislikes_log': 'Dislikes', 'comment_count_log': 'Comment Count', 'tagsCount': 'Tags Count', 'titleLength': 'Title Length', 'time_to_trend_hours': 'Hours to Trend', 'days_to trend': 'Days to Trend'})
corr_matrix = data_NG.corr(method='pearson')

# Create a graph object
G = nx.Graph()

# Add nodes to the graph
nodes = list(corr_matrix.columns)
G.add_nodes_from(nodes)

# Add edges to the graph
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        corr_val = corr_matrix.iloc[i, j]
        if corr_val > 0:
            G.add_edge(nodes[i], nodes[j], color='green', weight=abs(corr_val))
        elif corr_val < 0:
            G.add_edge(nodes[i], nodes[j], color='red', weight=abs(corr_val))

# Define edge colors based on the sign of correlation coefficient
edge_colors = [G[u][v]['color'] for u, v in G.edges()]

# Define node positions
pos = nx.shell_layout(G)

# Define node colors
node_colors = ['#FFE1E1']*len(G.nodes())

# Update node colors based on node degree
for node in G.nodes():
    degree = G.degree[node]
    if degree < 3:
        node_colors[nodes.index(node)] = 'green'

# Define Cytoscape elements
elements = []
for edge in G.edges():
    elements.append({'data': {'source': edge[0], 'target': edge[1], 'weight': G.get_edge_data(edge[0], edge[1])['weight']}, 'style': {'line-color': G.get_edge_data(edge[0], edge[1])['color'], 'width': abs(G.get_edge_data(edge[0], edge[1])['weight'])}})
for node in G.nodes():
    elements.append({'data': {'id': node}, 'position': {'x': pos[node][0], 'y': pos[node][1]}, 'style': {'background-color': node_colors[nodes.index(node)], 'label': node}})


In [ ]:
page_2_layout = dbc.Container([
            dbc.Row([
                    #dbc.Col(html.H1("YouTube Data Analysis",style={"color": "#7CBD1E"},className="text-center"), width=12),
                    ]),

            dbc.Row([
                    html.Br(),
                    html.Br(),
                    dbc.Col(html.Div("Hello there! Welcome to my project on Youtube Data Analysis."), width=12),
                    html.Br(),
                    html.Br(),
                    ]),

            dbc.Row([
                    dbc.Col(html.H2("No of Categories : 14",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    dbc.Col(html.H2("Years of Analysis: 2021, 2022",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    dbc.Col(html.H2("Country: US, IN",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    html.Br(),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div("First we can start with sunburst chart & tree map to get the brief overview of the data."), width=12),
                    html.Br(),
                    dbc.Col(html.Label('Select year:'), width=2),
                    dbc.Col(dcc.Dropdown(id='year-dropdown-sb', options=year_options, value=df['year'].min()), width=2),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='sunburst-chart'), width=6),
                    dbc.Col(dcc.Graph(id='treemap'), width=6),
                    html.Br(),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div("Select options to view stats",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    dbc.Col(html.Label('Select a Metric & Country:'), width=2),
                    dbc.Col(dcc.Dropdown(id='metric-dropdown', options=dropdown_options, value='view_count'), width=2),
                    dbc.Col(html.Label('Select Country:'), width=2),
                    dbc.Col(dcc.RadioItems(id='country-radio',options=country_options, value='US'), width=2),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='channel-metrics'), width=6),
                    dbc.Col(dcc.Graph(id='channel-metrics2'), width=6),
                    #dbc.Col(dcc.Graph(id='line-graph'), width=6),
                    #dbc.Col(dcc.Graph(figure = {}, id="my-graph"), width=5),
                    ]),
            dbc.Row([
                    dbc.Col(html.Label('Select a Year & Country:'), width=1),
                    dbc.Col(dcc.Dropdown(id='year1', options=year_options, value='2022'), width=1),
                    dbc.Col(dcc.RadioItems(id='country-radio2',options=country_options, value='US'), width=1),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='channel-metrics3'), width=6),
                    dbc.Col(dcc.Graph(id='channel-metrics4'), width=6),
                    ]),
            dbc.Row([
                    dbc.Col(html.Label("Select the Category for video counts",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=2),
                    dbc.Col(dcc.Dropdown(id='category-dropdown1', options=[{'label': i, 'value': i} for i in df['category'].unique()], value=df['category'].unique()[0]),width=2),
                    dbc.Col(html.Label("Select the options for Word Cloud",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=3),
                    dbc.Col(dcc.Dropdown(id='wc-image-dropdown', options=dropdown_options_wc, value=list(img_paths_wc.values())[0]),width=2),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='channel-metrics5'), width=4),
                    dbc.Col(html.Img(id='selected-image-wc', width='100%',height ='100%'), width=8),
                    ]),
])


geomaps_page_layout = dbc.Container([
            dbc.Row([
                    #dbc.Col(html.H1("YouTube Data Analysis",style={"color": "#7CBD1E"},className="text-center"), width=12),
                    ]),

            dbc.Row([
                    dbc.Col(html.Div("Hello there! Welcome to my project on Youtube Data Analysis. In this Page we will see Geo maps representing country Data."), width=12),
                    html.Br(),
                    html.Br(),
                    dbc.Col(html.Div("Select the Category to see video count in country",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    dbc.Col(dcc.Dropdown(id='category-dropdown-map', options=[{'label': i, 'value': i} for i in df['category'].unique()], value=df['category'].unique()[0]),width=2),
                    dbc.Col(dcc.Graph(id='video-count-map'), width=7),
                    ]),
            dbc.Row([
                    html.Label('2D Geo Map of User base'),
                    dcc.RadioItems(id='maps-radio',options=[{'label': 'Globe', 'value': 'Globe'}],value = 'Globe',),
                    dcc.Graph(id='map-div'),
                    #dbc.Col(dcc.Graph(id='globe-graph'), width=7),
                    ]),

])


stats_page_layout = dbc.Container([
            dbc.Row([
                    #dbc.Col(html.H1("YouTube Data Analysis",style={"color": "#7CBD1E"},className="text-center"), width=12),
                    ]),

            dbc.Row([
                    dbc.Col(html.Div("In this Page we will see Box Plots, Violin Plots, Swarm Plots, Heatmaps, scatterplot, "), width=12),
                    ]),

            dbc.Row([
                    #dbc.Col(html.H2("No of Categories : 14",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    #dbc.Col(html.H2("Years of Analysis: 2021, 2022",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    #dbc.Col(html.H2("Country: US, IN",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div("Box Plot",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    dbc.Col(html.Label('Select Country:'), width=2),
                    dbc.Col(dcc.RadioItems(id='country-radio1',options=country_options, value='US'), width=2),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='box-plot'), width=6),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div("Violin Plot",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    dbc.Col(html.Label('Select Country:'), width=2),
                    dbc.Col(dcc.RadioItems(id='country-radio2',options=country_options, value='US'), width=2),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='violin-plot'), width=6),
                    ]),
             dbc.Row([
                    #dbc.Col(html.Div("Heat Map",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=4),
                    #dbc.Col(html.Label('Select Country:'), width=2),
                    #dbc.Col(dcc.RadioItems(id='country-radio3',options=country_options, value='IN'), width=2),
                    ]),
            dbc.Row([
                    #dbc.Col(dcc.Graph(id='heat-map1'), width=6),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div("Histogram",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=2),
                    dbc.Col(html.Label('Select Bins:'), width=2),
                    dbc.Col(dcc.RadioItems(id='bins',options=bin_options, value=150), width=2),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='histogram1'), width=12),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div("KDE View Counts",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=2),
                    dbc.Col(html.Label('Select Bandwidth:'), width=2),
                    dbc.Col(dcc.RadioItems(id='bandwidth01',options=bandwidth_options, value=0.3), width=2),
                    dbc.Col(html.Div("Density Contour Plot",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=2),
                    dbc.Col(html.Label('Select Country:'), width=2),
                    dbc.Col(dcc.RadioItems(id='country1',options=country_options_contour, value='US'), width=2),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='kde'), width=6),
                    dbc.Col(dcc.Graph(id='2DcontourPlot1'), width=6),
                    ]),

])


ScatterPlot_page_layout = dbc.Container([
            dbc.Row([
                    html.Br(),
                    html.Br(),
                    dbc.Col(html.Div("Scatter Plot",style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=6),
                    dbc.Col(html.Div('Correlation Matrix Heatmap',style={"font-size": "18px", "color": "#7CBD1E"}, className="text-center"), width=6),
                    html.Br(),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(html.Label('Select a category & y value:'), width = 6),
                    dbc.Col(html.Label('Select a category for Heatmap:'), width = 6),
            ]),
            dbc.Row([
                    dbc.Col(dcc.Dropdown(id='category-dropdown',options=[{'label': cat, 'value': cat} for cat in df['category'].unique()], value=df['category'].unique()[0]), width=2),
                    dbc.Col(dcc.Dropdown(id='y-value-dropdown',options=[{'label': col.capitalize(), 'value': col} for col in ['likes_log', 'dislikes_log', 'comment_count_log', 'tagsCount', 'titleLength', 'time_to_trend_hours', 'hour_of_day', 'days_to trend']],value='likes'),width = 2),
                    dbc.Col(html.Button('Create Scatter Plot', id='scatter-plot-button'),width = 2),

                    dbc.Col(dcc.Dropdown(id='category-dropdown-hm',options=[{'label': c, 'value': c} for c in categories],value=categories[0]), width =3),
                    #dbc.Col(),
                    html.Br(),

                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='scatter-plot'), width=6),
                    dbc.Col(dcc.Graph(id='heatmap-graph'),width=6),
                    html.Br(),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(html.Label('3D Scatter Plot of Views, Likes and Comments by Category'), width = 6),
                    ]),
             dbc.Row([
                    dbc.Col(html.Label('Select a category:'), width = 3),
                    dbc.Col(dcc.Dropdown(id='category-dropdown-3D',options=[{'label': c, 'value': c} for c in categories],value=categories[0]), width = 3),
                    html.Br(),
                     ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='scatterplot-3D'), width=6),
                    dbc.Col(dcc.Graph(id='heatmap-dht', figure=fig_dht), width=6),
                    html.Br(),
                    html.Br(),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(html.Label('Hypothesis Testing for View Count Distribution between USA and India'), width = 6),
                    dbc.Col(dcc.Dropdown(id='category-dropdown-ht',options=[{'label': 'Music', 'value': 'Music'},{'label': 'News & Politics', 'value': 'News & Politics'}],value='Music'), width = 3),
                    #dbc.Col(),
                    ]),
            dbc.Row([
                    dbc.Col(dcc.Graph(id='histogram-graph-ht'), width=6),
                    #dbc.Col(),
                    ]),
])



networkGraph_page_layout = html.Div([
    html.H1('Network Graph Representing Tags from Top Pet channel in US', style={'textAlign': 'center'}), # Add title
    cyto.Cytoscape(
        id='cytoscape-graph',
        elements=elements,
        layout={'name': 'circle'},
        style={
            'width': '100%',
            'height': '500px',
            'background-color': '#f5f5f5', # color of the background
        }
    )
])


network_page2_layout = dbc.Container([
            dbc.Row([
                    html.Br(),
                    html.Br(),
                    dbc.Col(html.Div("Network Graph of Tags from Top Pet channel in US",style={"font-size": "18px", "color": "#FDF6F0"}, className="text-center"), width=6),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div("This is a circular network graph that displays the tags used in videos in the 'Pets & Animals'\
                    category and 'US' country. Nodes represent unique tags, edges indicate tags that tend to co-occur in the same video.\
                    From the graph, we can gain insights into co-occurring tags, popular tags, and influential tags, which can help\
                    content creators understand which tags are related, commonly used, and worth targeting. \
                    For Example in this graph, we can see that the tag 'my dog reacts'  \
                    has been used many times. And content creators can target for more such 'how my dog reacts' videos."), width=4),

                    dbc.Col(html.Div([
                        cyto.Cytoscape(id='cytoscape-graph',elements=elements1,layout={'name': 'concentric'},
                        style={'width': '100%','height': '500px','background-color': '#f5f5f5',})
                            ]), width = 8,
                           )
                    ]),
            dbc.Row([
                    html.Br(),
                    html.Br(),
                    dbc.Col(html.Div("Network Graph for Correalation between Metrics",style={"font-size": "18px", "color": "#FDF6F0"}, className="text-center"), width=6),
                    html.Br(),
                    ]),
            dbc.Row([
                    dbc.Col(html.Div([
                        cyto.Cytoscape(id='correlation-graph',elements=elements,layout={'name': 'circle'},
                        style={'width': '100%','height': '500px','background-color': '#f5f5f5',})
                            ]), width = 6,
                           ),
                    dbc.Col(html.Div("The correlation network graph shows the relationship between variables \
                    in a dataset by displaying the correlation coefficients as edges between nodes (variables).\
                    In this case, the nodes represent the variables'Views', 'Likes', 'Dislikes', 'Comments', 'Tag Count',\
                    'Title Length', 'Hours to Trend', and 'Days to Trend'. \
                    The edges between nodes indicate the strength of the correlation between two variables.\
                    A positive correlation between two variables is indicated by a green edge, \
                    while a negative correlation is indicated by a red edge.\
                    The thickness of the edge represents the strength of the correlation,\
                    with thicker edges representing stronger correlations.\
                    For example, in this particular graph, we can see that there is a strong positive correlation\
                    between 'Views' and 'Likes', as well as between 'Likes' and 'Comments'.\
                    On the other hand, there is a weak negative correlation \
                    between 'Title Length' and 'Days to Trend'"), width=6)
                    ]),
])


mlc_page_layout = dbc.Container([
            dbc.Row([
                    html.Br(),
                    dbc.Col(html.Div("Linear Regression Model for Predicting Likes",style={"font-size": "18px", "color": "#FDF6F0"}, className="text-center"), width=6),
                    html.Br(),
                    ]),
            dbc.Row([
                    html.Br(),
                    dbc.Col(html.Label('Enter View to predict likes'), width = 3),
                    dbc.Col(dcc.Input(id='new-views', type='number', placeholder='Enter views for new video'), width = 3),
                    dbc.Col(html.Button(id='submit-button', n_clicks=0, children='Predict'), width = 2),
                    html.Br(),
                    ]),
            dbc.Row([
                    html.Br(),
                    html.Br(),
                    dbc.Col(dcc.Graph(id='likes-predictions-graph'), width = 8),
                    ]),
            dbc.Row([
                    html.Br(),
                    dbc.Col(html.Div("K-Means Clustering",style={"font-size": "18px", "color": "#FDF6F0"}, className="text-center"), width=6),
                    html.Br(),
                    ]),
            dbc.Row([
                    html.Br(),
                    dbc.Col(html.Label('Below are the options to form clusters, where the default value is given by optimal no.of clusters by silhouette method is 4'), width = 6),
                    dbc.Col(dcc.RadioItems(options=[3, 4, 5], value=4, id='k-value'), width = 3),
                    html.Br(),
                    ]),
            dbc.Row([
                    html.Br(),
                    html.Br(),
                    dbc.Col(dcc.Graph(figure = {}, id="k-means-graph"), width = 8),
                    ]),
])


In [ ]:
#Define app callback for Treemap
@app.callback(
    Output('treemap', 'figure'),
    Input('year-dropdown-sb', 'value')
)
def generate_treemap(year):
    # filter data based on year
    filtered_data_tm = df[df['year'] == year]
    fig = px.treemap(filtered_data_tm, path=['category', 'country'], values='view_count')
    fig.update_layout(
        paper_bgcolor='#313131',  # Set paper color to white
        plot_bgcolor='#313131',  # Set background color to light gray
        font=dict(color='#FFE898'),
        title='Treemap: View Counts by Category and Country'
    )
    return fig


#Define app callback for Clustering
@app.callback(
    Output('k-means-graph', 'figure'),
    Input('k-value','value')
    )
def clustering_graph(kval):
    # Filter rows with categoryId == 'Music' and get 85 video only
    music_df = df[df['category'] == 'Music']
    less_than_50m = music_df[music_df['view_count'] < 50000000].head(30)
    between_50_100m = music_df[(music_df['view_count'] >= 50000000) & (music_df['view_count'] < 100000000)].head(20)
    between_100_150m = music_df[(music_df['view_count'] >= 100000000) & (music_df['view_count'] < 150000000)].head(10)
    between_150_200m = music_df[(music_df['view_count'] >= 150000000) & (music_df['view_count'] <= 200000000)].head(10)
    between_200_250m = music_df[(music_df['view_count'] >= 200000000) & (music_df['view_count'] < 250000000)].head(8)
    between_250_300m = music_df[(music_df['view_count'] >= 250000000) & (music_df['view_count'] <= 300000000)].head(7)
    combined_df1 = pd.concat([less_than_50m, between_50_100m, between_100_150m,between_150_200m,between_200_250m,between_250_300m])
    X = combined_df1[['view_count', 'likes']]
    kmeans = KMeans(n_clusters=kval)
    kmeans.fit(X)
    fig = go.Figure()
    fig.update_layout(
        plot_bgcolor='white',
        width=800,
        height=400
        )
    colors = ['red', 'blue','green','orange','yellow']
    fig.add_trace(go.Scatter(x=combined_df1['view_count'], y=combined_df1['likes'], mode='markers', marker=dict(color=kmeans.labels_, colorscale=colors, opacity=0.5, line=dict(color='grey',width=1))))
    for i in range(kval):
        centroid = kmeans.cluster_centers_[i]
        distances = np.linalg.norm(X[kmeans.labels_ == i] - centroid, axis=1)
        radius = np.percentile(distances, 90)
        fig.add_shape(type='circle',
                      xref='x', yref='y',
                      x0=centroid[0]+radius, y0=centroid[1]+radius, x1=centroid[0]-radius, y1=centroid[1]-radius,
                      line_color='#FCC2FC')
        fig.add_trace(go.Scatter(x=[centroid[0]], y=[centroid[1]], mode='markers', marker=dict(symbol='x', color='#AACB73', size=10)))
        #print(f"Cluster {i+1} centroid: ({centroid[0]:.2f}, {centroid[1]:.2f}), radius: {radius:.2f}")
    fig.update_layout(xaxis_title="Views", yaxis_title="Likes", title=f"K-Means Clustering Results (k={kval})",paper_bgcolor='#313131', plot_bgcolor='#313131',font=dict(color='#FF008E'))
    return fig



#Define app callback for Machine Learning
@app.callback(
    dash.dependencies.Output('likes-predictions-graph', 'figure'),
    dash.dependencies.Input('submit-button', 'n_clicks'),
    dash.dependencies.State('new-views', 'value')
)
def update_graph(n_clicks, new_views):
    # Prepare the data
    X = df['view_count'].values.reshape(-1, 1)  # Views
    y = df['likes'].values.reshape(-1, 1)  # Likes

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Fit the Linear Regression model to the data
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    # Make predictions using the Linear Regression model
    y_pred = regressor.predict(X_test)

    # Calculate the accuracy score
    accuracy_score = regressor.score(X_test, y_test)
    accuracy_score = accuracy_score*100
    #print("Accuracy score:", accuracy_score)

    # Predict likes for the new video
    new_views = np.array([[new_views]])  # Views of a new video
    new_likes = regressor.predict(new_views)  # Predicted likes
    #print("Predicted likes:", new_likes[0][0])

    # Create scatter plot for test data
    trace_test = go.Scatter(
        x=X_test.flatten(),
        y=y_test.flatten(),
        mode='markers',
        name='Test Data',
        marker=dict(
            color='#C3F8FF',
            opacity=0.7,
            line=dict(
                color='blue',  # Set edge line color to white
                width=1  # Set edge line width to 1
            )
        )
    )

    # Create line plot for regression line
    trace_reg = go.Scatter(
        x=X_test.flatten(),
        y=y_pred.flatten(),
        mode='lines',
        name='Regression Line',
        line=dict(
            color='blue'
        )
    )

    # Add a marker for the predicted value
    trace_pred = go.Scatter(
        x=[new_views.flatten()[0]],
        y=[new_likes.flatten()[0]],
        mode='markers',
        name='Predicted Likes',
        marker=dict(
            color='red',
            size=10
        ),
        text=f'Predicted Likes: {new_likes.flatten()[0]:.2f}'
    )

    # Create annotation for predicted score
    annotation = go.Annotation(
        x=new_views.flatten()[0],
        y=new_likes.flatten()[0],
        xref='x',
        yref='y',
        text=f'Predicted Likes: {new_likes.flatten()[0]:.2f}',
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-40
    )

    # Add all traces and annotation to figure
    fig = go.Figure(data=[trace_test, trace_reg, trace_pred], layout=go.Layout(
        title=f'Linear Regression Model for Predicting Likes (Accuracy score: {accuracy_score:.2f}%, Predicted likes: {new_likes.flatten()[0]:.2f})',
        xaxis=dict(title='Views', color='#FFFFFF'),
        yaxis=dict(title='Likes', color='#FFFFFF'),
        annotations=[annotation],
        plot_bgcolor='#313131',
        paper_bgcolor='#313131',
        font=dict(color='#99FEFF')
    ))

    # Display the figure
    return fig



# define the app callback for 3D Scatter Plot
@app.callback(
    Output('scatterplot-3D', 'figure'),
    [Input('category-dropdown-3D', 'value')]
)
def update_scatterplot(category):
    # filter the dataframe based on the category input
    filtered_df_3d = df[df['category'] == category]

    # create the 3D scatter plot using Plotly Express
    fig = px.scatter_3d(filtered_df_3d, x='views_log', y='likes_log', z='comment_count_log', color='category',size_max=10)

     # update background color
    fig.update_layout(scene=dict(bgcolor='#313131'),paper_bgcolor = '#313131')

    # update font color
    fig.update_layout(font=dict(color='#FF008E'))

    # update marker color and size
    fig.update_traces(marker=dict(color='#FF008E', size=2))

    # update x-axis label
    fig.update_layout(scene=dict(xaxis_title='Views',yaxis_title='Likes',zaxis_title='Comments'))

    # update title
    fig.update_layout(title=dict(text='3D Scatter Plot of Views, Likes and Comments'))

    return fig


# Define callback for Word Cloud
@app.callback(
    dash.dependencies.Output('selected-image-wc', 'src'),
    [dash.dependencies.Input('wc-image-dropdown', 'value')]
)
def update_image_src(value):
    return value

# Define callback to update sunburst chart based on selected year
@app.callback(
    Output('sunburst-chart', 'figure'),
    [Input('year-dropdown-sb', 'value')]
)
def generate_sunburst_chart(year):
    # filter data based on year
    filtered_data_sb = df[df['year'] == year]

    # create sunburst chart
    fig = px.sunburst(filtered_data_sb,
                      path=['category', 'country', 'GenericChannelName'],
                      color_continuous_scale='Pinkyl')

    # customize layout
    fig.update_layout(
        title={
            'text': f'Visualizing the Breakdown of Categories, Countries, and Channel Names ({year})',
            'font': {'size': 11}
        },
        #height=400,
        #width=600,
        plot_bgcolor='#313131',  # set background color of plot area
        paper_bgcolor='#313131',  # set background color of paper area (outside plot area)
        font=dict(color='#FFE898')
    )

    # display chart
    return fig


# Define the callback function to update the graph1
@app.callback(
    Output('channel-metrics', 'figure'),
    Input('metric-dropdown', 'value'),
    Input('country-radio', 'value')
)
def update_graph(selected_metric, country):
    # Filter by country (if provided)
    if country:
        df_filtered = df[df['country'] == country]
    else:
        df_filtered = df

    # Calculate the selected metric for each category
    category_metric = df_filtered.groupby(['category', 'GenericChannelName'])[selected_metric].mean().reset_index()

    # Create a bar chart
    fig = px.bar(category_metric, x=selected_metric, y='GenericChannelName', color='category',
                 title='Barplot: Visualizing {} by Category'.format(selected_metric.capitalize()))

    fig.update_layout(plot_bgcolor='#313131',
                      paper_bgcolor='#313131',
                      font=dict(color='#99FEFF')
                     )

    return fig


# Define the callback function to update the graph2
@app.callback(
    Output('channel-metrics2', 'figure'),
    Input('metric-dropdown', 'value'),
    Input('country-radio', 'value')
)
def update_graph2(selected_metric, country):

    # Filter by country (if provided)
    if country:
        df_filtered = df[df['country'] == country]
    else:
        df_filtered = df

    # Calculate the selected metric for each category
    category_metric = df_filtered.groupby(['category', 'GenericChannelName','month'])[selected_metric].mean().reset_index()

    # Create a bar chart
    fig = px.line(category_metric, x='month', y= selected_metric, color='category', markers=True,
             title=f'Line Plot Visualizing {selected_metric.capitalize()} by Month')

    fig.update_layout(plot_bgcolor='#313131',
                      paper_bgcolor='#313131',
                      font=dict(color='#99FEFF')
                     )

    return fig

# Define the callback function to update the graph3
@app.callback(
    Output('channel-metrics3', 'figure'),
    Input('year1', 'value'),
    Input('country-radio2', 'value')
)
def monthly_videocount_graph(year, country):

    # Filter by country (if provided)
    if country:
        df_filtered = df[df['country'] == country]
    else:
        df_filtered = df

    # Filter the DataFrame by year
    df_year = df_filtered[df_filtered['year'] == year]

    # Count the number of videos published on each month_name for each category
    day_count = df_year.groupby(['category', 'month_name']).size().reset_index(name='count')

    # Create a bar chart
    fig = px.bar(day_count, x='month_name', y='count', color='category',
                 title=f'Bar plot: Videos Published by Month in {year} for {country}')

    fig.update_layout(plot_bgcolor='#313131',
                      paper_bgcolor='#313131',
                      font=dict(color='#99FEFF')
                     )

    return fig

# Define the callback function to update the graph4
@app.callback(
    Output('channel-metrics4', 'figure'),
    Input('year1', 'value'),
    Input('country-radio2', 'value')
)
def daily_videocount_graph(year, country):
    # Filter by country (if provided)
    if country:
        df_filtered = df[df['country'] == country]
    else:
        df_filtered = df

    # Filter the dataframe to the selected year
    df_year = df_filtered[df_filtered['year'] == year]

    # Count the number of videos published on each day for each category
    day_count = df_year.groupby(['category', 'pushblishDayName']).size().reset_index(name='count')

    # Create a bar chart
    fig = px.bar(day_count, x='pushblishDayName', y='count', color='category',
                 title='Bar plot Videos published by Day in ' + str(year))

    fig.update_layout(plot_bgcolor='#313131',
                      paper_bgcolor='#313131',
                      font=dict(color='#99FEFF')
                     )

    return fig

# Define the callback function to update the graph5
@app.callback(
    Output('channel-metrics5', 'figure'),
    Input('category-dropdown1', 'value')
)
def donut_chart(category):
    filtered_df = df[df['category'] == category]
    filtered_df['view_count_millions'] = filtered_df['view_count'] / 1000000
    fig = px.pie(filtered_df, names='published_part_of_day', values='view_count_millions', hole=0.5,
                 opacity=0.8, color_discrete_sequence=px.colors.qualitative.G10,
                 hover_data={'view_count_millions': ':,.2f'})
    fig.update_traces(hovertemplate='%{label}: %{value:.2f} million views')
    fig.update_layout(paper_bgcolor='#313131',  # Set paper color to white
                      title = 'Donut Chart: View Count By Part of day',
                      plot_bgcolor='#313131',  # Set background color to light gray
                      font=dict(color='#FF008E'))
    return fig


# Define the app callback
@app.callback(
    Output('video-count-map', 'figure'),
    Input('category-dropdown-map', 'value')
)
def plot_video_count_by_country(category):
    # Create a dataframe with country and video_count columns for the given category

    df_country = df.groupby(['category', 'country']).count().reset_index()[['category', 'country', 'video_id']]
    df_country['country'] = df_country['country'].replace('US', 'USA')
    df_country['country'] = df_country['country'].replace('IN', 'IND')
    df_country = df_country[df_country['category'] == category][['country', 'video_id']]

    # Create a choropleth map with video_count as the color scale
    fig = px.choropleth(df_country,
                        locations='country',
                        locationmode='ISO-3',
                        color='video_id',
                        title=f'Geo Map: Video Count by Country for Category {category}',
                        color_continuous_scale='YlOrRd',  # Set the color scale
                        range_color=[100, 1000],
                        #facet_col='category'
                        ) # Set the legend title

    # Set the legend title
    fig.update_layout(coloraxis=dict(colorbar=dict(title="Video Count")))

    # Set the legend range from 100 to 1000
    fig.update_layout(coloraxis_colorbar=dict(x=0.85, len=0.75, y=0.5, thickness=10))

    # Update the layout
    fig.update_layout(
        paper_bgcolor='#313131',  # Set paper color to white
        #plot_bgcolor='#313131',  # Set background color to light gray
        font=dict(color='#FF008E'),
        width=1000,  # Set the width of the plot to 800 pixels
        height=600,  # Set the height of the plot to 800 pixels
        geo=dict(
            showframe=True,  # Hide the frame around the map
            showcoastlines=True,  # Hide the coastlines
            projection_type='natural earth', # Set the map projection
            showocean=True,
            oceancolor='#B9E9FC',
            scope='world',  # Adjust the map size and position
           )
        )
    return fig


#Function call back for globe graph
@app.callback(
    Output('map-div', 'figure'),
    Input('maps-radio', 'value')
)
def display_globe(x1):
        input_element = x1
        fig = px.choropleth(df1, locations='country_code', color='users_count',
                            hover_name='Country',
                            title='Geo Map: Top 20 Countries with the most YouTube users as of 2023 (in millions)',
                            #projection='robinson',
                            projection='orthographic',
                            color_continuous_scale='YlGnBu')
        # Set the legend range from 100 to 1000
        fig.update_layout(coloraxis_colorbar=dict(x=0.85, len=0.75, y=0.5, thickness=10,title='Users Count'))

        fig.update_layout(
              paper_bgcolor='#313131',  # Set paper color to white
              #plot_bgcolor='#313131',  # Set background color to light gray
              font=dict(color='#FF008E'),
              width=1000,  # Set the width of the plot to 800 pixels
              height=600,  # Set the height of the plot to 800 pixels
              geo=dict(
                  showframe=True,  # Hide the frame around the map
                  showcoastlines=True,  # Hide the coastlines
                  #projection_type='natural earth', # Set the map projection
                  showocean=True,
                  oceancolor='#B9E9FC',
                  scope='world',  # Adjust the map size and position
              )
        )
        # Show the plot
        return fig

# Define the callback function to update the box plot
@app.callback(
    Output('box-plot', 'figure'),
    Input('country-radio1', 'value')
)
def plot_box_by_country(country):
    # filter the DataFrame by country
    df_country_box = df[df['country'] == country]

    # create the box plot
    fig = px.box(df_country_box, x='category', y='time_to_trend_hours', color='category')
    fig.update_layout(title_text=f"Box Plot of Time to Trend for {country}", title_font_size=18,
                      xaxis_title="Category", xaxis_title_font_size=14,
                      yaxis_title="Time to Trend (hours)", yaxis_title_font_size=14,
                      width=1250, height=500,
                      paper_bgcolor='#313131', plot_bgcolor='#313131', font=dict(color='#FF008E'))
    return fig

# Define the callback function to update the violin
@app.callback(
    Output('violin-plot', 'figure'),
    Input('country-radio2', 'value')
)
def plot_violin_by_country(country):
    # filter the DataFrame by country
    df_country_box = df[df['country'] == country]

    # create the box plot
    fig = px.violin(df_country_box, x='category', y='days_to trend',box=True, color='category')
    fig.update_layout(title_text=f"Violin Plot of Days to Trend for {country}", title_font_size=18,
                      xaxis_title="Category", xaxis_title_font_size=14,
                      yaxis_title="Days to Trend", yaxis_title_font_size=14,
                      width=1250, height=500,
                      paper_bgcolor='#313131', plot_bgcolor='#313131', font=dict(color='#FF008E'))
    return fig

# Define the callback function to update the heatmap
@app.callback(
    Output('heat-map1', 'figure'),
    Input('country-radio3', 'value')
)
def plot_heatmap_by_country(country):
    df_country = df[df['country'] == country]
    heatmap_duration = df_country.groupby(['category','days_to trend']).count()['video_id'].unstack().clip(upper=300)

    fig = px.imshow(heatmap_duration, x=heatmap_duration.columns, y=heatmap_duration.index, color_continuous_scale='PiYg')
    fig.update_layout(title_text=f"Category v/s Trending Duration for {country}", title_font_size=18,
                      xaxis_title="Days to Trend", xaxis_title_font_size=14,
                      yaxis_title="Category", yaxis_title_font_size=14,
                      width=1250, height=500,
                      paper_bgcolor='#313131', plot_bgcolor='#313131', font=dict(color='#FF008E'),
                      showlegend=False)
    return fig

# Define the callback function to update the Histogram
@app.callback(
    Output('histogram1', 'figure'),
    Input('bins', 'value')
)
def histo_sub_plots(bins):
    # Create subplots
    fig = make_subplots(rows=2, cols=2, subplot_titles=("Views", "Likes", "Dislikes", "Comments"))
    # Add traces for views
    fig.add_trace(go.Histogram(x=df['views_log'], nbinsx=bins, marker_color='#FF008E', opacity=0.5),row=1, col=1)
    # Add traces for likes
    fig.add_trace(go.Histogram(x=df['likes_log'], nbinsx=bins, marker_color='#FF008E', opacity=0.5),row=1, col=2)
    # Add traces for dislikes
    fig.add_trace(go.Histogram(x=df['dislikes_log'], nbinsx=bins, marker_color='#FF008E', opacity=0.5),row=2, col=1)
    # Add traces for comments
    fig.add_trace(go.Histogram(x=df['comment_count_log'], nbinsx=bins, marker_color='#FF008E', opacity=0.5),row=2, col=2)
    # Update layout
    fig.update_layout(
    title_text="Distribution of Views, Likes, Dislikes, and Comments",
    title_font_size=18,
    width=1250,
    height=600,
    paper_bgcolor='#313131',
    plot_bgcolor='#313131',
    font=dict(color='#FF008E')
    )
    return fig

# Define the callback function to update the KDE Graph
@app.callback(
    Output('kde', 'figure'),
    Input('bandwidth01', 'value')
)
def display_kde(bandwidth01):
    view_counts =df['views_log'].to_numpy()
    # Compute the optimal bandwidth for the KDE
    kde = KernelDensity(bandwidth=bandwidth01 * np.std(view_counts), kernel='gaussian')
    view_counts = view_counts.reshape((len(view_counts), 1))
    # Fit the model to the data
    kde.fit(view_counts)
    values = np.linspace(10, 20, 1000).reshape(-1, 1)
    # Compute the probability for the range of values
    log_probabilities = kde.score_samples(values)
    probabilities = np.exp(log_probabilities)
    utube = pd.DataFrame({'x': values.flatten(), 'y': probabilities})
    # Create the KDE plot using Plotly Express
    fig = px.line(utube, x='x', y='y', title=f'KDE Plot (Bandwidth={bandwidth01})',color_discrete_sequence=['#FF008E']).update_layout(xaxis_title="view_counts", yaxis_title="Density")
     # Update layout
    fig.update_layout(title_text="Distribution of Views",
        title_font_size=18,
        width=600,
        height=400,
        paper_bgcolor='#313131',
        plot_bgcolor='#313131',
        font=dict(color='#FF008E')
        )
    return fig


# Define the callback function to update the densityContourPlor Graph
@app.callback(
    Output('2DcontourPlot1', 'figure'),
    Input('country1', 'value')
)
def create_density_contour(country1=None):
    # Filter the data for the given country
    if country1:
        country_df = df[df['country'] == country1]
        title = f'Density Contour Plot of Views and Likes for {country1}'
    else:
        country_df = df
        title = 'Density Contour Plot of Views and Likes for All Countries'

    # Create the density contour plot
    fig = px.density_contour(country_df, x="views_log", y="likes_log", marginal_x="histogram", marginal_y="histogram",color_discrete_sequence=['#06FF00'],
                             #hover_data={'views_actual':':.2f', 'likes_actual':':.2f'},
                             labels={'views_log': 'Views', 'likes_log': 'Likes',
                                     'view_count': 'Views', 'likes': 'Likes'},
                             histnorm='probability density',
                             title=f'Density Contour Plot of Views and Likes',
                             width=800, height=600)
    fig.update_layout(
    title_font_size=18,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    width=600,
    height=400,
    paper_bgcolor='#313131',
    plot_bgcolor='#313131',
    font=dict(color='#FF008E')
    )

    return fig


# Define the callback for the scatter plot
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('scatter-plot-button', 'n_clicks'),
    State('category-dropdown', 'value'),
    State('y-value-dropdown', 'value')
)
def update_scatter_plot(n_clicks, category_value, y_value):
    # Filter the dataframe by the given category value
    filtered_df = df[df['category'] == category_value]

    # Create a scatter plot with the filtered dataframe
    fig = px.scatter(filtered_df, x='views_log', y=y_value, trendline='ols', color='category',
                     color_discrete_sequence=['#C32BAD'],)


    # Update the figure layout
    fig.update_layout(title=f'Scatter plot with linear regression line ({category_value} category)',
                      xaxis_title='View Count',
                      yaxis_title=y_value.capitalize(),  # Set y-axis title based on the y_value passed to the function
                      paper_bgcolor='#313131',  # Set paper color to white
                      plot_bgcolor='#313131',  # Set background color to light gray
                      font=dict(color='#C449C2'),
                      width=600,  # Set the width of the plot to 800 pixels
                      height=400,
                      )

    return fig


# Define the callback function that updates the graph based on the selected category
@app.callback(
    Output('heatmap-graph', 'figure'),
    Input('category-dropdown-hm', 'value')
)
def create_corr_heatmap(category):
    # Filter the dataframe by the given category value
    filtered_df1 = df[df['category'] == category]

    # Define the column names you want to use in the heatmap
    column_names = {'view_count': 'Views',
                    'likes': 'Likes',
                    'dislikes': 'Dislikes',
                    'comment_count': 'Comments',
                    'tagsCount': 'Tags Count',
                    'titleLength': 'Title Length',
                    'time_to_trend_hours': 'Hours to Trend',
                    'hour_of_day': 'Hour of Day',
                    'days_to trend': 'Days to Trend'}

    # Rename the columns in the filtered dataframe using the column_names dictionary
    filtered_df1 = filtered_df1.rename(columns=column_names)

    # Create the correlation matrix
    corr_matrix = filtered_df1[['Views', 'Likes', 'Dislikes', 'Comments', 'Tags Count', 'Title Length', 'Hours to Trend', 'Hour of Day', 'Days to Trend']].corr()

    # Create the heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns.values,
            y=corr_matrix.columns.values,
            colorscale='agsunset',
            colorbar=dict(title='Correlation')
    ))

    # Update the figure layout
    fig.update_layout(title=f'Correlation Matrix Heatmap for {category} category',
                      paper_bgcolor='#313131',  # Set paper color to white
                      plot_bgcolor='#313131',  # Set background color to light gray
                      font=dict(color='#C449C2'),
                      width=600,
                      height=400,
                      )

    # Show the figure
    return fig


# Define the callback function
@app.callback(Output('histogram-graph-ht', 'figure'),
              Input('category-dropdown-ht', 'value'))
def view_count_t_distribution(category):
    df_hist1 = df[(df['category'] == category) & (df['country'] == 'US')]['views_log']
    df_hist2 = df[(df['category'] == category) & (df['country'] == 'IN')]['views_log']

    fig = go.Figure()

    #Calculate the Degree of Freedom
    def degreesOfFreedom(X, Y):
        s1 = np.std(X,ddof=1)
        s2 = np.std(Y,ddof=1)
        s1_sq = (s1**2)
        s2_sq = (s2**2)
        n1 = len(X)
        n2 = len(Y)
        defr = ((s1_sq/n1)+(s2_sq / n2))**2 / (((s1_sq/n1)**2 / (n1 - 1)) + ((s2_sq/n2)**2 / (n2 - 1)))
        return(defr)

    n = degreesOfFreedom(df_hist1, df_hist2)
    dof = n
    alpha = 0.05
    prob = 1- alpha
    critical_t_value = t.ppf(prob, dof)

    stat, p = stats.ttest_ind(df_hist1, df_hist2, equal_var=False)

    if p > alpha:
        reject_h0 = False
        reject_h0_text = 'Fail to Reject Null Hypothesis'
    else:
        reject_h0 = True
        reject_h0_text = 'Reject Null Hypothesis'

    xs = np.linspace(t.ppf(0.001, dof), t.ppf(0.999, dof), 100)

    fig.add_trace(go.Scatter(x=xs, y=st.t.pdf(xs,dof), name="t-Distribution PDF", line_color='purple'))
    fig.add_shape(type='line', x0=critical_t_value, y0=0, x1=critical_t_value, y1=0.1, line=dict(color='yellow', width=2,))
    fig.add_shape(type='line', x0=-critical_t_value, y0=0, x1=-critical_t_value, y1=0.1, line=dict(color='yellow', width=2,))
    fig.add_trace(go.Scatter(x=[stat, stat], y=[0, p], mode='lines', name="t-obs value", line=dict(color='red', width=2,dash='dash')))
    fig.add_annotation(text="Accept Ho", x=0, y=max(st.t.pdf(xs,dof))*0.4, showarrow=False)
    fig.add_annotation(text="Reject Ho", x=abs(critical_t_value)+1, y=max(st.t.pdf(xs,dof))*0.03, showarrow=False)
    fig.add_annotation(text="Reject Ho", x=-(abs(critical_t_value)+1), y=-(max(st.t.pdf(xs,dof))*0.001), showarrow=False)


    fig.update_layout(
        xaxis_title="t-Score",
        yaxis_title="Probability",
        title=f"View Count Distribution for {category} in US vs IN ({reject_h0_text})",
        paper_bgcolor='#313131',
        plot_bgcolor='#313131',
        font=dict(color='white'),
        width=800,
        height=400
    )

    #fig.show()
    return fig

In [ ]:
navbar = html.Div([
          html.H1("YouTube Data Analysis",style={"color": "#7CBD1E"},className="text-center"),
          html.Nav([
                    html.A('Basic Plots', href='/page-2'),
                    html.A('Geo Maps', href='/page-4'),
                    html.A('Analysis Plots I', href='/page-5'),
                    html.A('Analysis Plots II', href='/page-6'),
                    html.A('Network Graphs', href='/page-8'),
                    html.A('Machine Learning Algorithm & Clustering', href='/page-9'),
                    #html.A('Page 3', href='/page-3'),
                    # Repeat the above for the other pages
                    ], className='navbar')])

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),navbar,
    html.Div(id='page-content')
])

#ScatterPlot_page_layout
# Define the callback to update the page content based on the URL
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-2':
        return page_2_layout
    #elif pathname == '/page-3':
        #return page_3_layout
    elif pathname == '/page-4':
        return geomaps_page_layout
    elif pathname == '/page-5':
        return stats_page_layout
    elif pathname == '/page-6':
        return ScatterPlot_page_layout
    elif pathname == '/page-8':
        return network_page2_layout
    elif pathname == '/page-9':
        return mlc_page_layout
    # Repeat the above for the other pages
    else:
        return '404 Page not found'

In [ ]:
if __name__ == '__main__':
    app.run_server( port=8059, host='127.0.0.1')
    app.run_server(debug=False)

#app.run_server(mode='inline', port=8058)